In [ ]:
%load_ext autoreload
%autoreload 2

import uproot
import awkward as ak

import matplotlib.pylab as plt
import numpy as np

import time

from hist import Hist

import babar_analysis_tools as bat

from analysis_variables import *

import myPIDselector

import pandas as pd
import seaborn as sns

In [ ]:
#####################################################################
# Where are we running this?
#####################################################################
## Bellis computer
topdir= "/home/bellis/babar_data/bnv_plambda"

## My laptop
#topdir= "/Users/josieswann/BaBar_analyses/BNV_pLambda/"
#####################################################################


#####################################################################
# Get the BNV data
#####################################################################
#data, data_collision = bat.load_datasets(topdir=topdir, subset='Run1')

#####################################################################
# Get the BNC data
#####################################################################
topdir= "/home/bellis/babar_data/bnv_plambda_bnc"
#data, data_collision = bat.load_datasets(topdir=topdir, BNC=True, subset='all')
data, data_collision = bat.load_datasets(topdir=topdir, BNC=True, subset='Run1')

In [ ]:
region_definitions

# Using the final functions

## Generate the numbers for the cut flow

In [ ]:
df = bat.get_numbers_for_cut_flow(data, region_definitions=region_definitions, tag=None)

df

In [ ]:
#mask = df['spmode'] == '-999'
#mask = df['spmode'] != '-999'
#########################################################################################################
mask = (df['spmode'] == '998') | (df['spmode'] == '1005')

fig, ax = plt.subplots(figsize=(12,4))
sns.scatterplot(data=df[mask],x='name', y='pct', s=100, hue='spmode', style='spmode', ax=ax)#, markers=False)
sns.lineplot(data=df[mask],x='name', y='pct', hue='spmode', style='spmode', legend=False)#, markers=False)

plt.xticks(rotation=270)
plt.yscale('log')
plt.ylim(0.005,200)

#df[mask]


In [ ]:
mask = df['spmode'] == '-999'

fig, ax = plt.subplots(figsize=(12,4))
sns.scatterplot(data=df[mask],x='name', y='pct', s=100, hue='spmode', style='spmode', ax=ax)#, markers=False)
sns.lineplot(data=df[mask],x='name', y='pct', hue='spmode', style='spmode', legend=False)#, markers=False)

plt.xticks(rotation=270)
#plt.yscale('log')
plt.ylim(0.005,110)

#df[mask]


In [ ]:
df_coll = bat.get_numbers_for_cut_flow(data_collision, region_definitions=region_definitions, tag=None)

fig, ax = plt.subplots(figsize=(12,4))

#df_collision.plot.scatter(x='name', y='pct', s=100)
sns.scatterplot(data=df_coll,x='name', y='pct', s=100,  color='k')
sns.lineplot(data=df_coll,x='name', y='pct', color='k', legend=False)

plt.xticks(rotation=270)
plt.yscale('log')

df_coll

## Get the cuts and use them in plots

In [ ]:
# Need to get the original duplicates mask for any other cuts we might generate outside the function
dcuts = bat.get_final_masks(data, region_definitions=region_definitions)

print([dcuts.keys()])
print()

for key in dcuts.keys():
    print(f'{key:3d} {dcuts[key]["name"]}')

In [ ]:
#mask_event = dcuts[3]['event']
mask_event = dcuts[2]['event'] & dcuts[3]['event'] & dcuts[4]['event']

#spmode = '-999'
spmode = '998'
#spmode = '1005'
#spmode = '1235'

mask_sp = data['spmode']==spmode
mask = mask_sp & mask_event

x = data[mask]['BpostFitMes']

plt.hist(ak.flatten(x), bins=100, range=(5.2,5.3))
;

In [ ]:
################################################################################
# Make the masks
#mask_event = dcuts[3]['event']
#mask_event = dcuts[4]['event']
mask_event = dcuts[1]['event']
#mask_event = dcuts[-1]['event']
#mask_event = dcuts[2]['event'] & dcuts[3]['event'] & dcuts[4]['event']

#spmode = '998'
#spmode = '1005'
spmode = '-999'
#spmode = '1235'

mask_sp = data['spmode']==spmode

mask = mask_sp & mask_event
################################################################################

# Make the plot
mes =    ak.flatten(data[mask]['BpostFitMes'])
DeltaE = ak.flatten(data[mask]['BpostFitDeltaE'])

bat.plot_mes_vs_DeltaE(mes, DeltaE, region_definitions=region_definitions, draw_signal_region=True, zoom=True, bins=25)

#print(len(mes),len(DeltaE))

## Other HISTOGRAMS 
all_hists = bat.create_empty_histograms(hist_defs)
x = ak.flatten(data[mask]['Lambda0_unc_Mass'])
weight = 1.0#weights[spmode]
all_hists['Lambda0_unc_Mass'].fill(var=x, SP= spmode, cuts= f"{0}", weight= weight)

plt.figure()
all_hists['Lambda0_unc_Mass'].project('var').plot(histtype="fill", color='red', label= spmode)
all_hists['Lambda0_unc_Mass'].project('var').plot(histtype="step", color='black')
plt.legend();

# Break

# Developing and testing the function

In [ ]:
##########################################################################################
def munge_mask_shapes(mask_larger, mask_smaller):

    # mask_larger will be bigger and have some Trues and Falses
    #
    # We want a mask the same size as mask_larger but where the Trues
    # have been replaces by the values in mask_smaller

    # Get the original indices
    idx = ak.local_index(mask_larger)
    
    # Make a mask of Falses. This will be the replacement mask
    mask = ak.zeros_like(mask_larger, dtype=bool)

    # It's a bit easier to work with numpy for this step
    mask = mask.to_numpy()

    # Get the indices where the larger mask is true and set it equal to the 
    # values of the smaller mask
    mask[idx[mask_larger]] = mask_smaller

    return mask

##########################################################################################
def get_final_masks(data_temp, region_definitions=region_definitions, tag="DEFAULT"):

    #dcuts = {"cut":[], "name":[], "event":[], "candidates":[]}
    dcuts = {}
    
    #########################################################################
    # First select events without duplicate candidates
    #########################################################################
    mask_event_duplicates= bat.get_duplicates_mask(data_temp)

    ##########################################################################
    # Redefine the data array
    # tO only use the events with one candidate for Lambda0 and B
    ##########################################################################
    data = data_temp[mask_event_duplicates]

    #dcuts['cut'].append(1)
    dcuts[1] = {}
    dcuts[1]["name"] = "cut duplicates"
    dcuts[1]["event"] = mask_event_duplicates
    dcuts[1]["candidates"] = None
    
    
    #########################################################################
    # Events with Mes and DeltaE
    #########################################################################
    mask_candidates_fit = bat.get_fit_mask(data, region_definitions)

    mask_event_fit_region = ak.any(mask_candidates_fit, axis=-1)

    # Munge the shape to ensure it is the same shape as the original data array
    mask_event_fit_region = munge_mask_shapes(mask_event_duplicates, mask_event_fit_region)

    # We need to make sure it is the same size as the original file
    
    dcuts[2] = {}
    dcuts[2]["name"] = "fitting region"
    dcuts[2]["event"] = mask_event_fit_region
    dcuts[2]["candidates"] = mask_candidates_fit
    
    
    #########################################################################
    # Cut on Lambda and number of candidates
    #########################################################################
    mask_candidates_lambda0, mask_event_nlambda0_and_nB = bat.get_lambda0_mask(data, region_definitions=region_definitions, \
                                                                    flightlenvar='Lambda0FlightLen')

    # Munge the shape to ensure it is the same shape as the original data array
    mask_event_nlambda0_and_nB = munge_mask_shapes(mask_event_duplicates, mask_event_nlambda0_and_nB)

    dcuts[3] = {} 
    dcuts[3]["name"] = "Lambda0 cuts / nB / nLambda"
    dcuts[3]["event"] = mask_event_nlambda0_and_nB
    dcuts[3]["candidates"] = mask_candidates_lambda0
    
    #########################################################################
    # PID cuts
    #########################################################################
    
    mask_bool_proton, mask_bool_pion, mask_bool_protonB = bat.PID_masks(data, \
                  lamp_selector='SuperLooseKMProtonSelection', \
                  lampi_selector='VeryTightKMPionMicroSelection', \
                  Bp_selector='SuperTightKMProtonSelection', \
                  verbosity=0)

    #mask_event = mask_event_nlambda0_and_nB & mask_event_fit_region

    # We can do this because we selected the data where there is only one candidate for th
    # B and Lambda0
    mask_pid =      mask_bool_proton & \
                    mask_bool_pion & \
                    mask_bool_protonB

    mask_event_pid = ak.any(mask_pid, axis=-1)

    # Munge the shape to ensure it is the same shape as the original data array
    mask_event_pid = munge_mask_shapes(mask_event_duplicates, mask_event_pid)
    
    dcuts[4] = {} 
    dcuts[4]["name"] = "PID cuts"
    dcuts[4]["event"] = mask_event_pid
    dcuts[4]["candidates"] = mask_pid
    

    #########################################################################
    # Antiproton
    #########################################################################

    pps = myPIDselector.PIDselector("p")
    selector_to_test = "TightKMProtonSelection"
    
    # Is MC?
    # Note that we use the previous cuts on PID and fitting region

    mask_event_no_antiprotons, ct = bat.build_antiproton_antimask(data, pps, selector_to_test, IS_MC=IS_MC, verbose=0)
    
    #mask_event = mask_no_antiprotons & mask_event_pid & mask_event_nlambda0_and_nB & mask_event_fit_region

    # Munge the shape to ensure it is the same shape as the original data array
    mask_event_no_antiprotons = munge_mask_shapes(mask_event_duplicates, mask_event_no_antiprotons)

    
    dcuts[6] = {}
    dcuts[6]["name"] = "antiproton cuts"
    dcuts[6]["event"] = mask_event_no_antiprotons
    dcuts[6]["candidates"] = None

    #########################################################################
    # Everything
    #########################################################################

    mask_event = mask_event_no_antiprotons & mask_event_pid & mask_event_nlambda0_and_nB & mask_event_fit_region

    dcuts[-1] = {}
    dcuts[-1]["name"] = "all"
    dcuts[-1]["event"] = mask_event
    dcuts[-1]["candidates"] = None
    
    return dcuts

############################################################################


In [ ]:
# Need to get the original duplicates mask for any other cuts we might generate outside the function
dcuts = get_final_masks(data)


In [ ]:
print([dcuts.keys()])
print()

for key in dcuts.keys():
    print(f'{key:3d} {dcuts[key]["name"]}')



In [ ]:
#spmode = '998'
#spmode = '991'

spmode = '-999'
#spmode = '1235'

mask_sp = data['spmode']==spmode
norg = len(data[mask_sp])

print(f"norg: {norg}")

for key in dcuts.keys():
        
    mask_event = dcuts[key]['event']
    #mask_event_duplicates = dcuts[1]['event']
    
    #if key==1:
    #    mask_event = mask_event[mask_event_duplicates]
    
    # Anticuts for the antiproton
    #elif key==6:
    #    mask_event = mask_event[mask_event_duplicates]

    mask_sp = data['spmode']==spmode

    #mask = mask_sp[mask_event_duplicates] & mask_event
    mask = mask_sp & mask_event

    x = ak.flatten(data[mask]['BpostFitMes'])

    print(f'{key:3d} {len(x):6d} {100*len(x)/norg:6.2f}% {dcuts[key]["name"]}')



In [ ]:
print(len(data))

In [ ]:
mask_event = dcuts[-1]['event']
mask_event_duplicates = dcuts[1]['event']

#spmode = '-999'
spmode = '998'
#spmode = '991'
#spmode = '1235'

mask_sp = data['spmode']==spmode
print(data[mask_sp]['lowerID'])


#idx = ak.local_index(mask_event_duplicates)
#mask_total = ak.zeros_like(mask_event_duplicates, dtype=bool)
#mask_total = mask_total.to_numpy()
#mask_total[idx[mask_event_duplicates]] = mask_event

print(len(mask_event), len(mask_event_duplicates), len(mask_sp), len(data), len(mask_total))
print(mask_event)
print(mask_event_duplicates)
print(mask_sp)

mask = mask_sp & mask_total
#x = data[mask]['lowerID']
x = data[mask]['lowerID']

lid = data['lowerID']

icount = 0
for i in x:
# This is a 1235
#for i in [-1326255355]:
# This is -999
#for i in [1023551586]:
    print(i)
    mask_lid = lid== i
    #print(len(mask_lid))
    #print(data[mask_lid]['spmode'])
    #print(data[mask_lid]['lowerID'])
    num = ak.local_index(data, axis=0)
    #print(lid)
    #print(num)
    print(num[mask_lid][0])
    
    icount += 1

    if icount>10:
        break

In [ ]:
mask_event = dcuts[3]['event']
mask_event_duplicates = dcuts[1]['event']

#spmode = '-999'
#spmode = '998'
spmode = '1235'

mask_sp = data['spmode']==spmode

#mask = mask_sp[mask_event_duplicates] & mask_event
mask = mask_sp & mask_event

x = data[mask]['BpostFitMes']

plt.hist(ak.flatten(x), bins=100, range=(5.2,5.3))

;
print(len(ak.flatten(x)))

x = data[mask]['lowerID']
print(x)

In [ ]:
##########################################################################################
def get_numbers_for_cut_flow(data, region_definitions=region_definitions,tag="DEFAULT", spmodes=None):

    # Check to make sure the user passed in some SP modes
    if spmodes is None:
        print("Will default to using all the spmodes in the file")
        spmodes = np.unique(data['spmode'].to_list())
        print(spmodes)
        print()

    # Need to get the original duplicates mask for any other cuts we might generate outside the function
    dcuts = get_final_masks(data, region_definitions=region_definitions)

    df_dict = {"cut":[], "name":[], "nevents":[], "pct":[], "tag":[]}

    #########################################################################
    # Go through all the cuts
    #########################################################################
    for spmode in spmodes:
        mask_sp = data['spmode']==spmode

        #########################################################################
        # Org
        #########################################################################
        norg = len(data[mask_sp])
    
        df_dict["cut"].append(0)
        df_dict["name"].append("org")
        df_dict["nevents"].append(norg)
        df_dict["tag"].append(spmode)
    
        print(f"{norg} events in original file")
    
        df_dict["pct"].append(100*norg/norg)

        for key in dcuts.keys():
            print(f'{key:3d} {dcuts[key]["name"]}')
    
            # Get the event mask
            mask_event = dcuts[key]['event']
            #mask_event_duplicates = dcuts[key]['event']

            #print(len(mask_sp), len(mask_event), len(mask_event_duplicates))

            mask = mask_sp & mask_event
            n = len(mask[mask])
    
            df_dict["cut"].append(key)
            df_dict["name"].append(dcuts[key]["name"])
            df_dict["nevents"].append(n)
            df_dict["tag"].append(spmode)
            df_dict["pct"].append(100*n/norg)


    df = pd.DataFrame.from_dict(df_dict)
    
    return df

############################################################################
#spmode = '-999'
#bkg_spmodes = ['998', '1005', '1235', '1237', '3981', '991']
#sig_spmodes = ['-999']

#spmodes = bkg_spmodes + sig_spmodes

df = None
df = get_numbers_for_cut_flow(data, region_definitions=region_definitions, tag=spmode)

#df['pct'] = 100*df['nevents'] / df['nevents'].iloc[0]
#df

In [ ]:
#mask = df['tag'] == '-999'
#mask = df['tag'] != '-999'
#########################################################################################################
mask = (df['tag'] == '998') | (df['tag'] == '1005')

fig, ax = plt.subplots(figsize=(12,4))
sns.scatterplot(data=df[mask],x='name', y='pct', s=100, hue='tag', style='tag', ax=ax)#, markers=False)
sns.lineplot(data=df[mask],x='name', y='pct', hue='tag', style='tag', legend=False)#, markers=False)

plt.xticks(rotation=270)
plt.yscale('log')
plt.ylim(0.005,110)

df[mask]


In [ ]:
#########################################################################################################
mask = (df['tag'] == '-999')

fig, ax = plt.subplots(figsize=(12,4))
sns.scatterplot(data=df[mask],x='name', y='pct', s=100, hue='tag', style='tag', ax=ax)#, markers=False)
sns.lineplot(data=df[mask],x='name', y='pct', hue='tag', style='tag', legend=False)#, markers=False)

plt.xticks(rotation=270)
#plt.yscale('log')
plt.ylim(0.01,110)


df[mask]

## Early developmental work

In [ ]:
##########################################################################################
def get_all_masks(data, region_definitions=region_definitions,tag="DEFAULT"):

    df_dict = {"cut":[], "name":[], "nevents":[], "tag":[]}

    #########################################################################
    # Org
    #########################################################################
    norg = len(data)

    df_dict["cut"].append(0)
    df_dict["name"].append("org")
    df_dict["nevents"].append(norg)
    df_dict["tag"].append(tag)

    #########################################################################
    # Duplicate candidates
    #########################################################################
    mask_event_duplicates= bat.get_duplicates_mask(data)
    
    mask_event = mask_event_duplicates
    
    nsingles = len(mask_event[mask_event])

    # Only use the events with one candidate for Lambda0 and B
    data = data[mask_event]

    df_dict["cut"].append(1)
    df_dict["name"].append("cut duplicates")
    df_dict["nevents"].append(nsingles)
    df_dict["tag"].append(tag)
    
    #########################################################################
    # Events with Mes and DeltaE
    #########################################################################
    fit_mask = bat.get_fit_mask(data, region_definitions)
    #print(fit_mask)
    mask_event_fit_region = ak.any(fit_mask, axis=-1)

    mask_event = mask_event_fit_region
    
    nfit_region = len(mask_event[mask_event])

    df_dict["cut"].append(2)
    df_dict["name"].append("fitting region")
    df_dict["nevents"].append(nfit_region)
    df_dict["tag"].append(tag)
    

    #########################################################################
    # Cut on Lambda and number of candidates
    #########################################################################
    mask_lambda0, mask_event_nlambda0_and_nB = bat.get_lambda0_mask(data, region_definitions=region_definitions, \
                                                                    flightlenvar='Lambda0FlightLen')

    mask_event = mask_event_nlambda0_and_nB

    nlambda0_cuts = len(mask_event[mask_event])

    df_dict["cut"].append(3)
    df_dict["name"].append("Lambd0 cuts / nB / nLambda")
    df_dict["nevents"].append(nlambda0_cuts)
    df_dict["tag"].append(tag)
    

    #########################################################################
    # Summary of cuts
    #########################################################################

    mask_event = mask_event_nlambda0_and_nB & mask_event_fit_region

    nsummary_0 = len(mask_event[mask_event])

    df_dict["cut"].append(4)
    df_dict["name"].append("fitting region and Lambd0 cuts / nB / nLambda")
    df_dict["nevents"].append(nsummary_0)
    df_dict["tag"].append(tag)
    

    #########################################################################
    # PID cuts
    #########################################################################
    
    mask_bool_proton, mask_bool_pion, mask_bool_protonB = bat.PID_masks(data, \
                  lamp_selector='SuperLooseKMProtonSelection', \
                  lampi_selector='VeryTightKMPionMicroSelection', \
                  Bp_selector='SuperTightKMProtonSelection', \
                  verbosity=0)

    mask_event = mask_event_nlambda0_and_nB & mask_event_fit_region

    # We can do this because we selected the data where there is only one candidate for th
    # B and Lambda0
    mask_pid =      mask_bool_proton & \
                    mask_bool_pion & \
                    mask_bool_protonB

    mask_event_pid = ak.any(mask_pid, axis=-1)
    mask_event = mask_event_pid
    
    npid = len(mask_event[mask_event])

    df_dict["cut"].append(5)
    df_dict["name"].append("PID cuts and prev")
    df_dict["nevents"].append(npid)
    df_dict["tag"].append(tag)
    

    #########################################################################
    # Antiproton
    #########################################################################

    pps = myPIDselector.PIDselector("p")
    selector_to_test = "TightKMProtonSelection"
    
    # Is MC?
    # Note that we use the previous cuts on PID and fitting region

    #mask_event_temp = mask_pid

    #print(len(data[mask_event_pid]))
    mask_no_antiprotons, ct = bat.build_antiproton_antimask(data, pps, selector_to_test, IS_MC=IS_MC, verbose=0)

    #print(len(mask_event_pid), len(mask_no_antiprotons), len(ct))
    #print(mask_no_antiprotons)
    #print(ct)
    
    mask_event = mask_no_antiprotons & mask_event_pid & mask_event_nlambda0_and_nB & mask_event_fit_region

    nantip = len(mask_event[mask_event])

    df_dict["cut"].append(6)
    df_dict["name"].append("antiproton and PID cuts and prev")
    df_dict["nevents"].append(nantip)
    df_dict["tag"].append(tag)
    
    
    #########################################################################
    # Print summaries
    #########################################################################    
    print(f"norg:          {norg}")
    print(f"nsingle:       {nsingles}")
    print(f"nfit_region:   {nfit_region}")
    print(f"nlambda0_cuts: {nlambda0_cuts}")
    print(f"nsummary_0:    {nsummary_0}")
    print(f"npid:          {npid}")
    print(f"nantip:        {nantip}")

    df = pd.DataFrame.from_dict(df_dict)
    
    return df

############################################################################
spmode = '-999'

bkg_spmodes = ['998', '1005', '1235', '1237', '3981', '991']
sig_spmodes = ['-999']

spmodes = bkg_spmodes + sig_spmodes

df = None
for i,spmode in enumerate(spmodes):
    mask_sp = data['spmode']==spmode
    
    data_sp = data[mask_sp]
    
    df_temp = get_all_masks(data_sp, region_definitions=region_definitions, tag=spmode)

    df_temp['pct'] = 100*df_temp['nevents'] / df_temp['nevents'].iloc[0]
    
    if i==0:
        df = df_temp.copy()
    else:
        df = pd.concat([df, df_temp], ignore_index=True)

df

In [ ]:
data_collision.show

In [ ]:
fit_mask = bat.get_fit_mask(data_collision, region_definitions)
print(len(data_collision))
print(fit_mask)
mask_event_fit_region = ak.any(fit_mask, axis=-1)

mask_event = mask_event_fit_region

nfit_region = len(mask_event[mask_event])

print(nfit_region)


In [ ]:
df

In [ ]:
mask = df['cut']== 6

df[mask].plot.scatter(x='tag', y='pct')

df[mask]

In [ ]:
#mask = df['tag'] == '-999'
#mask = df['tag'] != '-999'
#########################################################################################################
mask = (df['tag'] == '998') | (df['tag'] == '1005')

fig, ax = plt.subplots(figsize=(12,4))
sns.scatterplot(data=df[mask],x='name', y='pct', s=100, hue='tag', style='tag', ax=ax)#, markers=False)
sns.lineplot(data=df[mask],x='name', y='pct', hue='tag', style='tag', legend=False)#, markers=False)

plt.xticks(rotation=270)
plt.yscale('log')
plt.ylim(0.01,110)


#########################################################################################################
mask = (df['tag'] == '-999')

fig, ax = plt.subplots(figsize=(12,4))
sns.scatterplot(data=df[mask],x='name', y='pct', s=100, hue='tag', style='tag', ax=ax)#, markers=False)
sns.lineplot(data=df[mask],x='name', y='pct', hue='tag', style='tag', legend=False)#, markers=False)

plt.xticks(rotation=270)
#plt.yscale('log')
plt.ylim(0.01,110)



df[mask]

In [ ]:
df_temp = get_all_masks(data_collision, region_definitions=region_definitions, tag="0")

df_temp['pct'] = 100*df_temp['nevents'] / df_temp['nevents'].iloc[0]

df_collision = df_temp.copy()

fig, ax = plt.subplots(figsize=(12,4))

#df_collision.plot.scatter(x='name', y='pct', s=100)
sns.scatterplot(data=df_collision,x='name', y='pct', s=100,  color='k')
sns.lineplot(data=df_collision,x='name', y='pct', color='k', legend=False)

plt.xticks(rotation=270)
plt.yscale('log')

df_collision

In [ ]:
# Use it
# Cut on Lambda and number of candidates
mask_lambda0, mask_event_nlambda0_and_nB = bat.get_lambda0_mask(data_sp, region_definitions=region_definitions, \
                                                                flightlenvar='Lambda0FlightLen')

# Use only the data in the fitting region
fit_mask = bat.get_fit_mask(data_sp, region_definitions)

######################################

mask_bool_proton, mask_bool_pion, mask_bool_protonB = bat.PID_masks(data_sp, \
              lamp_selector='SuperLooseKMProtonSelection', \
              lampi_selector='VeryTightKMPionMicroSelection', \
              Bp_selector='SuperTightKMProtonSelection', \
              verbosity=0)


pps = myPIDselector.PIDselector("p")
selector_to_test = "TightKMProtonSelection"
# Is MC?
mask_no_antiprotons, ct = bat.build_antiproton_antimask(data_sp, pps, selector_to_test, IS_MC=IS_MC, verbose=0)
print(f"mask no antip: {len(mask_no_antiprotons)}    mask lambda0/nB: {len(mask_event_nlambda0_and_nB)}")

mask_event = mask_event_nlambda0_and_nB & mask_no_antiprotons


mask_pid =      mask_bool_proton[mask_event] & \
                mask_bool_pion[mask_event] & \
                mask_bool_protonB[mask_event]

mask_candidates = fit_mask[mask_event] & mask_lambda0[mask_event] & mask_pid 

#mask_candidates = fit_mask & mask_lambda0 & mask_pid 

# Make the plot
mes =    ak.flatten(data_sp[mask_event]['BpostFitMes'][mask_candidates])
DeltaE = ak.flatten(data_sp[mask_event]['BpostFitDeltaE'][mask_candidates])

bat.plot_mes_vs_DeltaE(mes, DeltaE, region_definitions=region_definitions, draw_signal_region=True, zoom=True, bins=25)

#print(len(mes),len(DeltaE))

## Other HISTOGRAMS 
all_hists = bat.create_empty_histograms(hist_defs)
x = ak.flatten(data_sp[mask_event]['Lambda0_unc_Mass'][mask_candidates])
weight = weights[spmode]
all_hists['Lambda0_unc_Mass'].fill(var=x, SP= spmode, cuts= f"{0}", weight= weight)

plt.figure()
all_hists['Lambda0_unc_Mass'].project('var').plot(histtype="fill", color='red', label= spmode)
all_hists['Lambda0_unc_Mass'].project('var').plot(histtype="step", color='black')
plt.legend();